<a href="https://colab.research.google.com/github/Christopher-jason/Text-Analysis/blob/main/Topic_modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python -m spacy download en_core_web_lg

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-02-01 15:02:18.269297: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 1.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [ ]:
import spacy
spacy.load('en_core_web_lg')
from spacy.lang.en import English
parser = English()

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [ ]:
def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.corpus import wordnet as wn

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma

In [ ]:
from nltk.stem.wordnet import WordNetLemmatizer
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

In [ ]:
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

In [ ]:
import sklearn
# Use 20newsgropups data
groups = [
    'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware',
    'comp.sys.mac.hardware', 'comp.windows.x', 'sci.space']
train_data = sklearn.datasets.fetch_20newsgroups(subset="train",
                                                 categories=groups)
print("Number of training posts in tech groups:", len(train_data.filenames))


Number of training posts in tech groups: 3529


In [ ]:
import re
text_data = []

In [ ]:
data = train_data.data

In [ ]:
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]


In [ ]:
for tmp_data in data:
  tokens = prepare_text_for_lda(tmp_data)
  text_data.append(tokens)

In [ ]:
from gensim import corpora
#Let's create data to the gensim format
dictionary = corpora.Dictionary(text_data)
corpus = [dictionary.doc2bow(text) for text in text_data]


In [ ]:
import gensim
# Now let's do Topic Modelling
NUM_TOPICS = 25
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(17, '0.015*"motif" + 0.012*"available" + 0.012*"version" + 0.011*"widget"')
(3, '0.031*"space" + 0.012*"launch" + 0.009*"satellite" + 0.006*"mission"')
(10, '0.040*"entry" + 0.018*"program" + 0.012*"section" + 0.010*"rule"')
(5, '0.054*"drive" + 0.014*"controller" + 0.012*"line" + 0.012*"subject"')
(18, '0.012*"line" + 0.012*"subject" + 0.011*"group" + 0.011*"organization"')
(9, '0.015*"organization" + 0.014*"subject" + 0.014*"line" + 0.014*"monitor"')
(15, '0.005*"flight" + 0.004*"wings" + 0.004*"keyboard" + 0.004*"edwards"')
(1, '0.018*"problem" + 0.017*"line" + 0.016*"subject" + 0.015*"organization"')
(7, '0.015*"subject" + 0.014*"line" + 0.013*"window" + 0.013*"organization"')
(20, '0.017*"would" + 0.015*"write" + 0.014*"line" + 0.014*"subject"')
(23, '0.024*"subject" + 0.024*"line" + 0.022*"organization" + 0.015*"windows"')
(0, '0.018*"visual" + 0.017*"widget" + 0.012*"default" + 0.012*"value"')
(11, '0.062*"driver" + 0.030*"windows" + 0.014*"diamond" + 0.012*"line"')
(2, '0.015*

In [ ]:
new_doc = 'Practical Bayesian Optimization of Machine Learning Algorithms'
new_doc = prepare_text_for_lda(new_doc)
new_doc_bow = dictionary.doc2bow(new_doc)
print(new_doc_bow)
print(ldamodel.get_document_topics(new_doc_bow))


[(46, 1), (253, 1), (528, 1), (701, 1), (1613, 1)]
[(3, 0.3448386), (23, 0.5018281)]


In [ ]:
!pip install pyLDAvis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 43.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=e61729cd0b99bbf59259a31b2841bff7092995c2dc8de7dc7e0cf4b8da2d495d
  Stored in directory: /root/.cache/pip/wheels/90/61/ec/9dbe9efc3acf9c4e37ba70fbbcc3f3a0ebd121060aa593181a
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2344 sha256=7c784911872a87d58ac66b4f1b832036c0e7f1c5f40f4d20ae960580c0ccf2cc
  Stored in directory: /root/.cache/pip/wheels/14/25/f7/1cc0956978ae479e75140219088deb7a36f60459df242b1a72
Successfully built pyLDAvis sklearn


In [ ]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis  # don't skip this
import matplotlib.pyplot as plt

In [ ]:
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(ldamodel, corpus, dictionary)
vis

/usr/local/lib/python3.8/dist-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.116461  0.144587       1        1  11.716107
8      0.166419  0.147088       2        1   9.751743
7      0.205079  0.034640       3        1   8.426555
20     0.164788 -0.034361       4        1   8.026022
5      0.184107 -0.063432       5        1   7.495180
22     0.171437 -0.041742       6        1   6.906643
14     0.178733 -0.090204       7        1   6.142344
17     0.103269  0.186889       8        1   6.041594
1      0.170646 -0.101184       9        1   5.806507
23     0.140882 -0.083122      10        1   4.179107
19    -0.274430 -0.056296      11        1   3.663948
10     0.013184  0.191248      12        1   3.590177
9      0.055515 -0.110520      13        1   2.425318
11     0.043526 -0.096133      14        1   2.335942
6     -0.061109 -0.023999      15        1   1.651829
21    -0.067460  0.004428      16        1   1.508775
13    -0.092551  0.020277      17        1   1.488110
0     -0.152275  0.135319      18        1   1.406983
4     -0.208867  0.029424      19        1   1.373217
16    -0.080743 -0.029465      20        1   1.297721
18    -0.060743 -0.069235      21        1   1.177812
24    -0.249549 -0.009381      22        1   1.042301
2     -0.151512 -0.008197      23        1   1.008978
15    -0.183810 -0.028007      24        1   0.782516
12    -0.130996 -0.048622      25        1   0.754571, topic_info=            Term         Freq        Total Category  logprob  loglift
1683       drive  1388.000000  1388.000000  Default  30.0000  30.0000
526       window   881.000000   881.000000  Default  29.0000  29.0000
460       driver   618.000000   618.000000  Default  28.0000  28.0000
275      windows  1471.000000  1471.000000  Default  27.0000  27.0000
23         space  1444.000000  1444.000000  Default  26.0000  26.0000
...          ...          ...          ...      ...      ...      ...
725      running     8.132431   447.947730  Topic25  -5.7438   0.8780
892      anybody     6.411794   246.344883  Topic25  -5.9816   1.2382
273   university     7.117624  1556.111950  Topic25  -5.8771  -0.5006
209        write     6.615644  2187.147584  Topic25  -5.9503  -0.9141
611   difference     5.798866   199.202894  Topic25  -6.0820   1.3501

[1687 rows x 6 columns], token_table=       Topic      Freq                   Term
term                                         
22401     23  0.744417  +1.510.980.1958(pager
27018      7  0.217048                 -----+
27018     25  0.759669                 -----+
2977       1  0.090889               --------
2977       2  0.327199               --------
...      ...       ...                    ...
1396       1  0.238526               |_____|/
1396       9  0.742081               |_____|/
3872       7  0.305611            |||||||||||
3872      25  0.611221            |||||||||||
37357      7  0.922633     ~/.echo.${hostname

[5100 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 9, 8, 21, 6, 23, 15, 18, 2, 24, 20, 11, 10, 12, 7, 22, 14, 1, 5, 17, 19, 25, 3, 16, 13])